In [14]:
import pandas as pd
import joblib

In [15]:
all_good_cases = set()
all_bad_cases = set()
all_non_effective_cases = set()
all_okay_cases = set()

for split in ['train', 'eval', 'test']:
    print(10 * '----')
    print(split)
    main_df = pd.read_csv(f"basic_model/{split}.csv")
    print(main_df.shape)
    main_df.head(3)

    cbr_df = pd.read_csv(f"cbr_simcse_cases/{split}.csv")
    print(cbr_df.shape)
    cbr_df.head(3)

    # Just a sanity check for the order of the rows
    for i in range(main_df.shape[0]):
        if not cbr_df['masked_articles'][i].startswith(main_df['masked_articles'][i]):
            print("something's Wrong.")

    main_df.drop(columns=['masked_articles'], axis = 1, inplace = True)
    cbr_df.drop(columns=['masked_articles', 'true_labels'], axis = 1, inplace = True)
    cbr_df.rename({"predictions": "cbr_predictions"}, inplace=True, axis=1)

    all_df = pd.concat([main_df, cbr_df], axis = 1)

    bad_cases = all_df[(all_df['predictions'] == all_df['true_labels']) & (all_df['cbr_predictions'] != all_df['true_labels'])]['cbr'].tolist()
    good_cases = all_df[(all_df['predictions'] != all_df['true_labels']) & (all_df['cbr_predictions'] == all_df['true_labels'])]['cbr'].tolist()
    non_effective_cases = all_df[(all_df['predictions'] != all_df['true_labels']) & (all_df['cbr_predictions'] != all_df['true_labels'])]['cbr'].tolist()
    okay_cases = all_df[(all_df['predictions'] == all_df['true_labels']) & (all_df['cbr_predictions'] == all_df['true_labels'])]['cbr'].tolist()

    agg_good_cases = set()
    for good_case in good_cases:
        agg_good_cases.update(set(good_case.split('</sep>')))

    agg_bad_cases = set()
    for bad_case in bad_cases:
        agg_bad_cases.update(set(bad_case.split('</sep>')))
    
    agg_non_effective_cases = set()
    for non_effective_case in non_effective_cases:
        agg_non_effective_cases.update(set(non_effective_case.split('</sep>')))

    agg_okay_cases = set()
    for okay_case in okay_cases:
        agg_okay_cases.update(set(okay_case.split('</sep>')))



    # print(len(agg_good_cases), len(agg_good_cases - agg_bad_cases))
    # print(len(agg_bad_cases), len(agg_bad_cases - agg_good_cases))
    
    # agg_good_cases = agg_good_cases - agg_bad_cases
    
    # print(len(agg_good_cases), len(agg_good_cases - agg_bad_cases))
    # print(len(agg_bad_cases), len(agg_bad_cases - agg_good_cases))
    
    if split != 'test':
        all_good_cases.update(agg_good_cases)
        all_bad_cases.update(agg_bad_cases)
        all_okay_cases.update(agg_okay_cases)
        all_non_effective_cases.update(agg_non_effective_cases)

----------------------------------------
train
(1849, 3)
(1849, 4)
----------------------------------------
eval
(300, 3)
(300, 4)
----------------------------------------
test
(300, 3)
(300, 4)


In [16]:
print("all_good_cases", len(all_good_cases))
print("all_bad_cases", len(all_bad_cases))
print("all_non_effective_cases", len(all_non_effective_cases))
print("all_okay_cases", len(all_okay_cases))

all_good_cases 48
all_bad_cases 87
all_non_effective_cases 172
all_okay_cases 996


In [17]:
print('cases to included', len(set.union(all_good_cases, all_okay_cases, all_non_effective_cases) - all_bad_cases))
print('cases not to included', len(all_bad_cases - all_non_effective_cases - all_good_cases))

cases to included 985
cases not to included 71


In [18]:
all_new_good_cases = set.union(all_good_cases, all_okay_cases, all_non_effective_cases) - all_bad_cases
all_new_bad_cases = all_bad_cases - all_non_effective_cases - all_good_cases
print('cases to included', len(all_new_good_cases))
print('cases not to included', len(all_new_bad_cases))

cases to included 985
cases not to included 71


In [19]:
joblib.dump(all_new_good_cases, '../all_good_cases.joblib')
joblib.dump(all_new_bad_cases, '../all_bad_cases.joblib')

['../all_bad_cases.joblib']